In [1]:
import cv2
import numpy as np
import os
import pickle
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

In [2]:
def load_images(folder_path, label, image_size=(64, 64)):
    images = []
    labels = []
    for filename in os.listdir(folder_path):
        img = cv2.imread(os.path.join(folder_path , filename))
        if img is not None:
            img = cv2.resize(img, image_size)  # Resize
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
            images.append(img.flatten())  # Flatten to 1D array
            labels.append(label)
    return images, labels

In [3]:
cat_folder = 'Cats'
dog_folder = 'Dogs'

cat_images, cat_labels = load_images(cat_folder, label=0)  # Label cats as 0
dog_images, dog_labels = load_images(dog_folder, label=1)  # Label dogs as 1

In [4]:
X = np.array(cat_images + dog_images)
y = np.array(cat_labels + dog_labels)


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [6]:
svm = SVC(kernel='rbf', gamma='scale', C=1.0)  # RBF kernel is good for image classification
svm.fit(X_train, y_train)

# Test the model
y_pred = svm.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.6144578313253012
Classification Report:
               precision    recall  f1-score   support

           0       0.69      0.29      0.41        38
           1       0.60      0.89      0.71        45

    accuracy                           0.61        83
   macro avg       0.64      0.59      0.56        83
weighted avg       0.64      0.61      0.57        83



In [7]:
param_grid = {
    'C': [0.1, 1, 10],
    'gamma': ['scale', 0.1, 0.01],
    'kernel': ['linear', 'rbf']
}

grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=2)
grid.fit(X_train, y_train)

print("Best Parameters:", grid.best_params_)
best_model = grid.best_estimator_

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time=   0.0s
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time=   0.0s
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time=   0.0s
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time=   0.0s
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time=   0.0s
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time=   0.0s
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time=   0.0s
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time=   0.0s
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time=   0.0s
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time=   0.0s
[CV] END ....................C=0.1, gamma=0.1, kernel=linear; total time=   0.0s
[CV] END ....................C=0.1, gamma=0.1, k

In [8]:
with open('cat_dog_svm_model.pkl', 'wb') as f:
    pickle.dump(best_model, f)

with open('cat_dog_svm_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)